# VOCS v1 data structure 

In [1]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

/Users/chrisonian/Code/GitHub/Xopt/xopt/base.py:21: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _futures = pd.Series()


In [2]:
?VOCS

Init signature:
VOCS(
    *,
    variables: Dict[str, types.ConstrainedListValue] = None,
    constraints: Dict[str, types.ConstrainedListValue] = None,
    objectives: Dict[str, xopt.vocs.ObjectiveEnum] = None,
    constants: Dict[str, Any] = None,
    linked_variables: Dict[str, str] = None,
) -> None
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/Code/GitHub/Xopt/xopt/vocs.py
Type:           ModelMetaclass
Subclasses:     


In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables=None)

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': None}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs


True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives['c']  == ObjectiveEnum.MAXIMIZE

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": null}'

In [10]:
vocs.schema()

{'title': 'VOCS',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'definitions': {'ConstraintEnum': {'title': 'ConstraintEnum',
   'description': 'An enumeration.',
   'enum': ['LESS_THAN', 'GREATER_THAN'],
   'type': 'string'},
  'ObjectiveEnum': {'title': 'ObjectiveEnum',
   'description': 'An enumeration.',


# Objective Evaluation

In [11]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data['c']  = data['a'] + data['b']
data['d']  = data['a'] - data['b']
data['e']  = data['a']*2 + data['b']*2
data['f']  = data['a']*2 - data['b']*2
data.index = np.arange(len(data)) + 5 # custom index
data


,a,b,g,c,d,e,f
5,508.537966,-0.718434,1234,507.819533,509.256400,1015.639066,1018.512800
6,710.816141,-0.000329,1234,710.815812,710.816469,1421.631624,1421.632939
7,569.146564,0.526577,1234,569.673141,568.619987,1139.346281,1137.239974
8,964.769449,-0.421030,1234,964.348418,965.190479,1928.696837,1930.380958
9,392.900549,0.447793,1234,393.348343,392.452756,786.696686,784.905512
10,260.261294,-0.022368,1234,260.238926,260.283661,520.477852,520.567323
11,293.087758,-0.308430,1234,292.779328,293.396188,585.558655,586.792377
12,222.131815,0.418115,1234,222.549930,221.713700,445.099861,443.427400
13,968.683593,0.144278,1234,968.827871,968.539315,1937.655742,1937.078630
14,432.373134,-0.906655,1234,431.466479,433.279789,862.932958,866.559578


In [12]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [13]:
?form_objective_data

Signature: form_objective_data(objectives: Dict, data, prefix='objective_')
Docstring:
Use objective dict and data (dataframe) to generate objective data (dataframe)

Weights are applied to convert all objectives into mimimization form.

Returns a dataframe with the objective data intented to be minimized.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [14]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-507.819533,509.256400
6,-710.815812,710.816469
7,-569.673141,568.619987
8,-964.348418,965.190479
9,-393.348343,392.452756
10,-260.238926,260.283661
11,-292.779328,293.396188
12,-222.549930,221.713700
13,-968.827871,968.539315
14,-431.466479,433.279789


In [15]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-507.819533,509.256400
6,-710.815812,710.816469
7,-569.673141,568.619987
8,-964.348418,965.190479
9,-393.348343,392.452756
10,-260.238926,260.283661
11,-292.779328,293.396188
12,-222.549930,221.713700
13,-968.827871,968.539315
14,-431.466479,433.279789


In [16]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-507.81953288,  509.2563999 ],
       [-710.81581223,  710.81646938],
       [-569.67314066,  568.619987  ],
       [-964.34841848,  965.19047903],
       [-393.34834277,  392.45275578],
       [-260.2389258 ,  260.28366126],
       [-292.77932775,  293.3961883 ],
       [-222.54993033,  221.71370003],
       [-968.82787094,  968.53931511],
       [-431.46647887,  433.27978899]])

In [17]:
?form_constraint_data

Signature: form_constraint_data(constraints: Dict, data, prefix='constraint_')
Docstring:
Use constraint dict and data (dataframe) to generate constraint data (dataframe)
A constraint is satisfied if the evaluation is < 0.

Returns a dataframe with the constraint data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [18]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1013.639066,-1018.512800
6,1419.631624,-1421.632939
7,1137.346281,-1137.239974
8,1926.696837,-1930.380958
9,784.696686,-784.905512
10,518.477852,-520.567323
11,583.558655,-586.792377
12,443.099861,-443.427400
13,1935.655742,-1937.078630
14,860.932958,-866.559578


In [19]:
?form_feasibility_data

Signature: form_feasibility_data(vocs, data, prefix='feasibility_')
Docstring:
Use constraint dict and data to identify feasible points in the the dataset.

Returns a dataframe with the feasibility data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [20]:
vocs.feasibility_data(data)

,feasibility_e,feasibility_f,feasibility
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False
